In [1]:
from test_data import gapped_sequences, sequences, motif_1
from benchmark import Benchmark
from skbio.sequence import genetic_code, DNA
from qiime_default_reference import get_reference_sequences
skbio_benchmark = Benchmark("scikit-bio master")
motif_1_regex = '(' + motif_1 + ')'


In [2]:
!cat test_data.py

from qiime_default_reference import get_template_alignment, get_reference_sequences

from skbio import SequenceCollection

gapped_sequences = [(s.id, str(s)) for s in SequenceCollection.read(get_template_alignment())][:500]

sequences = [(s.id, str(s)) for s in SequenceCollection.read(get_reference_sequences())][:500]

motif_1 = "GGTGCAAGCCGGTGGAAACA"


In [3]:
from skbio import DNA
skbio_seqs = [DNA(seq, id=str(id)) for id_, seq in sequences]
skbio_gapped_seqs = [DNA(seq, id=str(id)) for id_, seq in gapped_sequences]

In [86]:
@skbio_benchmark
def object_creation():
    for id_, seq in sequences:
        DNA(seq, id=id_, validate=False)

1000 loops, best of 3: 576 µs per loop


In [87]:
@skbio_benchmark
def object_creation_validate():
    for id_, seq in sequences:
        DNA(seq, id=id_)

100 loops, best of 3: 2.24 ms per loop


In [24]:
@skbio_benchmark
def read_fasta_file():
    DNA.read(get_reference_sequences())

1000 loops, best of 3: 789 µs per loop


In [89]:
@skbio_benchmark
def reverse_complement():
    for s in skbio_seqs:
        s.reverse_complement()

1 loops, best of 3: 1.53 s per loop


In [11]:
@skbio_benchmark
def search_for_motif():
    for seq in skbio_seqs:
        list(seq.slices_from_regex(motif_1_regex))

100 loops, best of 3: 18.1 ms per loop


In [14]:
#John
sgc = genetic_code(1)
@skbio_benchmark
def translate():
    for seq in skbio_seqs:
        sgc.translate(seq, 1)
 

1 loops, best of 3: 6.07 s per loop


In [92]:
# @skbio_benchmark
# def reverse_translate():
#     pass

In [93]:
# @skbio_benchmark
# def filter_fasta_to_no_gaps():
#     pass

In [94]:
@skbio_benchmark
def degap_all():
    for s in skbio_gapped_seqs:
        s.degap()

100 loops, best of 3: 8.2 ms per loop


In [15]:
@skbio_benchmark
def kmer_count_5():
    for seq in skbio_seqs:
        seq.kmer_frequencies(5)

1 loops, best of 3: 19.6 s per loop


In [16]:
@skbio_benchmark
def kmer_count_25():
    for seq in skbio_seqs:
        seq.kmer_frequencies(25)

1 loops, best of 3: 19.2 s per loop


In [17]:
@skbio_benchmark
def validate_chars():
    pass

            

100 loops, best of 3: 17 ms per loop


In [ ]:
seq = skbio_seqs[0]


In [98]:
@skbio_benchmark
def filter_invalid_seqs():
    pass

In [99]:

# @skbio_benchmark
# def rc_find_motif():
#     pass

In [100]:
# @skbio_benchmark
# def expand_degenerates():
#     pass

In [101]:
# @skbio_benchmark
# def gc_content():
#     pass

In [9]:
@skbio_benchmark
def find_motif_in_gapped():
    for seq in skbio_gapped_seqs:
        list(seq.slices_from_regex(motif_1_regex, ignore=seq.gaps()))

10 loops, best of 3: 67.4 ms per loop


In [103]:
skbio_benchmark.record("skbio.csv")